This notebook with call the necessary functions to pass a data frame to an LLM and have it make suggestions for the output:

In [29]:
#Library import for loading df
from llm_data_checker import read_df


#incase of error with file cache uncomment below command 
#importlib.reload(llm_data_checker)

data = read_df("data_test/Uncleaned_DS_jobs.csv")


#run  source .venv/bin/activate to activate virtual environment first
#data.head()

Run this cell to check basic stats of df:

In [30]:
from llm_data_checker import df_checker

check_data = df_checker(data)


#write out the stats to external directory
with open("stats_output/stats.txt", "w") as f:
    for section_name, section_value in check_data.items():
        # Write a header for this section
        f.write(f"===== {section_name} =====\n")
        
        # Write the actual data
        f.write(str(section_value))
        
        # Add spacing
        f.write("\n\n")

Anonymiser: 

Prompt builder:

In [31]:
from pathlib import Path 

#point to directoy holding prompt files
framework_txt = Path("frameworks")

#points to the directory holding the df stats 
stats_txt = Path("stats_output" )

#read the actual files themselves
system_template = (framework_txt / "prompt.txt").read_text()


func_test_suite = (framework_txt / "func_test_suite.txt").read_text()
function_format = (framework_txt / "function_format.txt").read_text()
stats = (stats_txt / "stats.txt").read_text()
reasoning = (framework_txt / "reasoning.txt").read_text()


#build the actual prompt 

prompt = system_template.format(
    func_test_suite=func_test_suite,
    function_format=function_format,
    stats=stats,
    reasoning=reasoning,
)

#write out the stats to external directory
with open("final_prompt/combined_prompt.txt", "w") as f:
    f.write(prompt)



In [32]:
combined_prompt = Path("final_prompt/combined_prompt.txt").read_text()
print(len(combined_prompt))

6546


API call out 


In [37]:
from cerebras.cloud.sdk import Cerebras
from pathlib import Path
import os

client = Cerebras(api_key=os.environ["CEREBRAS_API_KEY"])

completion = client.chat.completions.create(
    model="gpt-oss-120b",
    messages=[
        {"role": "system", "content": combined_prompt},
        {
            "role": "user",
            "content": """
Produce TWO outputs exactly as specified in the system prompt.

Return them in this format ONLY:

===== FUNC_TEST_SUITE =====
<full contents of func_test_suite.txt>

===== REASONING =====
<full contents of reasoning.txt>

Do NOT include any other text.
"""
        },
    ],
    max_completion_tokens=8192,  # Increase for complex analysis
    temperature=0.0,
    top_p=0.95,  # Add this
    frequency_penalty=0.0,  # Reduce repetition
    presence_penalty=0.0,   # Encourage completeness
)

output = completion.choices[0].message.content

Write out results from LLM:

In [38]:
from pathlib import Path



out_path = Path("llm_suggestions/llm_output.txt")
out_path.write_text(output, encoding="utf-8")

6970